# Notebook to parse FB data recieved from Facebook Graph Explorer API

## Import libraries to assist in analysis

In [ ]:
import pandas as pd
import pickle
import numpy
import matplotlib.pyplot as plt
import os
%matplotlib inline

In [2]:
def readData(filename):
    """
    Method reads the facebook Graph API dump data already stored on the
    disk in pickle format. The data is in JSON format
    :param filename : filename for the data stored on the disk
    :return : None
    """
    with open(filename, "rb") as fileHandler:
        data = pickle.load(fileHandler)
        return data
    
fbDataFile = "../data/JSON_from_FB/10207648390994439"
data = readData(fbDataFile)

In [3]:
data.keys()

dict_keys(['likes', 'work', 'about', 'id', 'favorite_athletes', 'tagged_places', 'location', 'gender', 'photos', 'education', 'events', 'relationship_status', 'family', 'birthday', 'friends', 'sports', 'name', 'favorite_teams', 'hometown'])

In [4]:
data['friends']['data'][0]['id']

'859854448375'

In [5]:
def parse1Level(parentFriend, user, userJson):
    """
    Function to extract friend list of given user from the given userJson
    :param parendFriend : Integer. Friend of given user
    :param user : Integer as user_id e.g. 1975030262039
    :param userJson : in Dict format e.g. {'birthday': '11/13/1991',
                                            'education': [{'id': '1975030262039',
                                            'school': {'id': '108546329176028', 'name': 'MPVM Ganga Gurukulam'},
                                            'type': 'High School',
                                            'year': {'id': '136328419721520', 'name': '2009'}}],
                                            friends: {friendList in the same format}}
    :return: List. Friend of given user extacted from userJson
    """
    friendList = set()
    try:
        for friend in userJson['friends']['data']:
            friendList.add(friend['id'])
        friendList.add(user)
        return list(friendList)[:]
    except Exception as e:
        # handling part if user has no friend, function return empty list
        return [parentFriend]
    

def parseFullJson(userJson, mode):
    """
    Function to extract friend list of given user from the given userJson
    :param userJson : in Dict format e.g. {'birthday': '11/13/1991',
                                            'education': [{'id': '1975030262039',
                                            'school': {'id': '108546329176028', 'name': 'MPVM Ganga Gurukulam'},
                                            'type': 'High School',
                                            'year': {'id': '136328419721520', 'name': '2009'}}],
                                            friends: {friendList in the same format}}
    :return: None
    """
    visitedUser=set()
    parentUser = userJson['id']
    
    with open("../data/MasterAdjList", mode) as fileHandler:        
        friendList = parse1Level(parentUser, parentUser, userJson)
        writeLine(parentUser, friendList, fileHandler, visitedUser)
        visitedUser.add(parentUser)
        for userJson1 in userJson['friends']['data']:
            user = userJson1['id']
            friendList = parse1Level(parentUser, user, userJson1)            
            writeLine(user, friendList, fileHandler,visitedUser) 
            visitedUser.add(user)
        try:
            for userJson1 in userJson['friends']['data']:
                parentUser = userJson1['id']
                for userJson2 in userJson1['friends']['data']:
                    user = userJson2['id']
                    friendList = parse1Level(parentUser, user, userJson2)
                    writeLine(user, friendList, fileHandler, visitedUser)
                    visitedUser.add(user)
        except Exception as e:
            # handling part if user's friend has no friend
            pass
            
            
        # one level remaining
    
def writeLine(user, friendList, fileHandler, visitedUser):
    """
    Functions write 1 row of the adjList in the givenFile
    :param user : Integer . e.g 1975030262039
    :param friendList : List . List of user friends
    :param fileHandler : File handler in which entries are to be made
    :param visitedUser : set . Set of Integers that has already been written in the file
    :return : None
    """
    if user not in visitedUser:
        fileHandler.write(str(user)+" ")
        for friend in friendList:
            fileHandler.write(friend+" ")
        fileHandler.write("\n")
    return None

# friendList = parse1Level("10207648390994439", data)
# friendList

# parseFullJson(data, "w")
# parse1Level("Parentdummy", "dummy", data['friends']['data'][0]['friends']['data'][0])

In [8]:
def buildMasterAdjList(folderPath):
    arr = os.listdir(folderPath)
    flag = 0
    for file in arr:
        data = readData(folderPath+file)
        if flag==0:
            parseFullJson(data, "w")
            flag=1
        else:
            parseFullJson(data, "a")
    
buildMasterAdjList("../data/JSON_from_FB/")